In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Flatten, Dropout
from keras.datasets import mnist
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

if not 'savefig' in os.listdir():
    os.mkdir('savefig')

### 🐍mnistデータ読み込む🐍
- 読み込んだ後に画像の画素値を-1～1に正規化した

In [2]:
(X_train, _), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

batch_size = 256

half_batch = int(batch_size / 2)

11493376/11490434 [==============================] - 0s 0us/step


### 🐍最適化関数はAdamに🐍
- なんでAdamにしたか？[ここ](https://www.renom.jp/ja/notebooks/tutorial/basic_algorithm/adam/notebook.html)参照してください。
- とても長いので集約すると、現時点では最も適切なパラメータ更新ができる最適化手法として認められているから。

In [3]:
z_dim = 100

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

optimizer =Adam(0.0002, 0.5)

Instructions for updating:
Colocations handled automatically by placer.


### 🐍Generatorを設計🐍

In [0]:
noise_shape = (z_dim,)
generator = Sequential()

generator.add(Dense(256, input_shape=noise_shape))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.2))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(np.prod(img_shape), activation='tanh'))
generator.add(Reshape(img_shape))

### 🐍Discriminatorを設計🐍

In [0]:
img_shape = (img_rows, img_cols, channels)
discriminator = Sequential()

discriminator.add(Flatten(input_shape=img_shape))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(1, activation='sigmoid'))

In [0]:
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### 🐍combinedを設計🐍
- GeneratorとDiscriminatorをつなぎ合わせたものを作成しておく
- Generatorを学習させるときは、Generator単体ではなく、このCombinedを使って学習を行う

In [0]:
discriminator.trainable = False
combined = Sequential([generator, discriminator])
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

### 🐍学習🐍
- 規定回数ごとに画像を保存している
- ちょっとあとでもう少しわかりやすくします
- 最終的には、Discriminatorの正答率が50%になるくらいまで、Generatorに成長してもらいたい

In [13]:
for epoch in range(20000):
    
    # 以下、Discriminator学習
    # Generatorによる偽データの生成
    noise = np.random.normal(0, 1, (half_batch, z_dim))
    gen_imgs = generator.predict(noise)
    
    # バッチサイズの半数を教師データからピックアップ
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    imgs = X_train[idx]
    
    # discriminator学習
    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    
    # それぞれの損失関数を平均
    d_loss_mean = np.add(d_loss_real, d_loss_fake) * 0.5
    
    # 以下、Generator学習
    noise = np.random.normal(0, 1, (batch_size, z_dim))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
    
    if epoch % 500 == 0:
        
        row = 5
        columns = 5
        save_gen_imgs = gen_imgs[: row * columns]
        save_gen_imgs = 0.5 * save_gen_imgs + 0.5
        fig, axes = plt.subplots(row, columns)

        for r in range(row):
            for c in range(columns):
                axes[r, c].imshow(save_gen_imgs[r * 5 + c, :, :, 0], cmap='gray')
                axes[r, c].axis('off')

        plt.savefig('savefig/{}.png'.format(str(epoch)))
        plt.close()

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [10]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive
